
<span  style="float:right"><a href="http://moldesign.bionano.autodesk.com/" target="_blank" title="About">About</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://forum.bionano.autodesk.com/c/Molecular-Design-Toolkit" target="_blank" title="Forum">Forum</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://github.com/autodesk/molecular-design-toolkit/issues" target="_blank" title="Issues">Issues</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="http://bionano.autodesk.com/MolecularDesignToolkit/explore.html" target="_blank" title="Tutorials">Tutorials</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="http://autodesk.github.io/molecular-design-toolkit/" target="_blank" title="Documentation">Documentation</a></span>
</span>
![Molecular Design Toolkit](img/Top.png)
<br>


<center><h1>Tutorial 2: Playing with proteins</h1></center>

Here, you'll see how to build, visualize, and simulate a protein structure from the PDB.

In [ ]:
import moldesign as mdt
from moldesign import units as u

%matplotlib inline
import numpy as np
from matplotlib.pylab import *
try: import seaborn
except ImportError: pass

Contents
=======
---
 - [1. Download from PDB](#1.-Download-from-PDB)
 - [2. Strip water and assign forcefield](#2.-Strip-water-and-assign-forcefield)
 - [3. Set up energy model and minimize](#3.-Set-up-energy-model-and-minimize)
 - [4. Add integrator and run dynamics](#4.-Add-integrator-and-run-dynamics)
 - [5. Some simple analysis](#5.-Some-simple-analysis)

# 1. Download from PDB
In this example, we'll download `1YU8`, a structure of the Villin Headpiece.

In [ ]:
one_yu8 = mdt.from_pdb('1YU8')
one_yu8.draw()

In [ ]:
one_yu8

# 2. Strip water and assign forcefield

In [ ]:
headpiece = mdt.Molecule([res for res in one_yu8.residues if res.type == 'protein'])

In [ ]:
ff = mdt.forcefields.DefaultAmber()
protein = ff.create_prepped_molecule(headpiece)

# 3. Set up energy model and minimize

In [ ]:
protein.set_energy_model(mdt.models.OpenMMPotential, implicit_solvent='obc')

In [ ]:
mintraj = protein.minimize()

In [ ]:
mintraj.draw(display=False)

# 4. Add integrator and run dynamics

In [ ]:
protein.set_integrator(mdt.integrators.OpenMMLangevin,
                       temperature=300*u.kelvin,
                       timestep=2.0*u.fs,
                       frame_interval=2.0*u.ps)

In [ ]:
traj = protein.run(50*u.ps)

In [ ]:
traj.draw()

# 5. Some simple analysis
As in tutorial 1, tutorial objects permit a range of timeseries-based analyses.

In [ ]:
plot(traj.time, traj.kinetic_energy)
xlabel('time / %s' % u.default.time); ylabel('energy / %s' % u.default.energy)

In [ ]:
protein

In [ ]:
myres = protein.chains[0].residues['PHE47']
plot(traj.time, traj.dihedral(myres['CG'], myres['CB']).to(u.degrees))
title('dihedral angle vs time')
xlabel('time / %s' % u.default.time); ylabel('angle / degrees')

In [ ]:
plot(traj.time, traj.distance(myres['CG'], myres['CB']))
plt.title('bond length vs time')
xlabel('time / %s' % u.default.time); ylabel('distance / %s' % u.default.length)